In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np

In [3]:
def classNameToLabel(className):
    return {
        'Loans': 0,
        'Credit card': 1,
        'Promotion Campaign': 2,
        'Account Balance': 3,
        'E-banking': 4,
        'FX': 5,
        'Mortgages': 6,
        'Insurance': 7,
        'Transfer Services': 8
    }[className]

In [4]:
df = pd.read_csv('dataset.csv')
df['Class'] = df['Class'].apply(classNameToLabel)

df.head()

,Class,Script
0,0,點樣決定貸款額
1,0,有咩唔同還款方式比我揀
2,0,借貸期啫係咩意思同埋幾長先適合
3,0,申請貸款要交啲咩文件
4,0,每個月平息係咩意思


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Script'], df['Class'], shuffle=True, stratify=df['Class'])

In [57]:
#Bert Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_zh_preprocess/3") 
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/4",
    trainable=True)
    
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].

output = tf.keras.layers.Dropout(0.1)(pooled_output)
# x = tf.keras.layers.Dense(128, activation='relu')(pooled_output)
output = tf.keras.layers.Dense(9, activation='softmax')(output)

model = tf.keras.Model(inputs=text_input, outputs=output)

In [58]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_word_ids':   0           ['input_3[0][0]']                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [59]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [60]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
5/5 [==============================] - 166s 16s/step - loss: 3.0425 - accuracy: 0.0850
Epoch 2/10
5/5 [==============================] - 75s 15s/step - loss: 2.6016 - accuracy: 0.1046
Epoch 3/10
5/5 [==============================] - 78s 15s/step - loss: 2.4414 - accuracy: 0.1307
Epoch 4/10
5/5 [==============================] - 79s 16s/step - loss: 2.3392 - accuracy: 0.0915
Epoch 5/10
5/5 [==============================] - 89s 18s/step - loss: 2.3251 - accuracy: 0.1438
Epoch 6/10
5/5 [==============================] - 82s 16s/step - loss: 2.4026 - accuracy: 0.1307
Epoch 7/10
5/5 [==============================] - 81s 16s/step - loss: 2.2976 - accuracy: 0.1438
Epoch 8/10
5/5 [==============================] - 79s 15s/step - loss: 2.3435 - accuracy: 0.1438
Epoch 9/10
5/5 [==============================] - 86s 17s/step - loss: 2.2972 - accuracy: 0.0784
Epoch 10/10
5/5 [==============================] - 85s 17s/step - loss: 2.3145 - accuracy: 0.1242


In [61]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

2/2 - 55s - loss: 2.2027 - accuracy: 0.1569 - 55s/epoch - 27s/step

Test accuracy: 0.1568627506494522


In [49]:
p = model.predict(X_train)

In [62]:
model.save('nlp.h5')

In [63]:
my_reloaded_model = tf.keras.models.load_model(('nlp.h5'),custom_objects={'KerasLayer':hub.KerasLayer},compile=False)